In [5]:
import pandas as pd
import nibabel as nib
import numpy as np
import os
from dice import dice

from utils import organizeFolders, organizeFiles, openImage

In [12]:
# Main Folder Dir
fp = 'C:\\Users\\colli\\Desktop\\lesionInterfaces\\ISMRM'

# Pandas dataframe to store everything
"""
Shorthand meanings
    - 'vol' = original image volume
    - 'GT' = ground truth segmentation
    - 'MS' = Manual Segmentation
    - 'SC' = Smart Click Segmentation
"""
df = pd.DataFrame(columns=[
    # Patient ID
    'patient_id',

    # Voxel Count
    'vol_voxCnt',
    'GT_voxCnt',
    'MS_voxCnt',
    'SC_voxCnt',

    # Dice Score
    'GT-MS_dice',
    'GT-SC_dice',
    'MS_SC_dice',
])

# Maping file names to respective categories
file_map = {
    'vol' : 'dicom_vol',
    'GT' : 'GT_liver_cyst_vol',
    'MS' : 'DR_liver_cyst_vol',
    'SC' : 'DR_liver_cyst_smart_click_vol'}

inv_file_map = {v: k for k, v in file_map.items()}

In [13]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def findIndex(file_map, a):
    best_choice = None
    best_score = 0
    for k, v in file_map.items():
        if similar(a, k) > best_score:
            best_choice = similar(a, k)
            best_choice = k

    return best_choice

In [31]:
df = pd.DataFrame()
splitter = '\\'

# Get folder paths
folder_dir = organizeFolders(fp)

# Get file paths and info
for i in folder_dir:
    # Get File Paths
    file_dir = organizeFiles(i)
    patient_id = i.split(splitter)[-1]

    # Organize all numpy arrays into dictionary following the file_map
    np_files = {}
    np_files_vol = {}

    for j in file_dir:
        np_arr, volume = openImage(j)
        try:
            np_files[ inv_file_map[j.split(splitter)[-1].split('.nii')[0]] ] = np_arr
            np_files_vol[ inv_file_map[j.split(splitter)[-1].split('.nii')[0]] ] = volume
        except:
            print('Error: ', j)
            pass


    print(patient_id, '---', np_files_vol)

    # Start geting information
    ## voxCnt and volume

    temp = {}
    keyword = '_voxCnt'
    keyword2 = '_volume(in cm^3)'
    for k in np_files.keys():
        temp['patient_id'] = patient_id
        if k == 'vol':
            temp[k+keyword] = round(np_files[k].shape[0] * np_files[k].shape[1] * np_files[k].shape[2])
            temp[k+keyword2] = round(np_files_vol[k] * temp[k+keyword], 6)
        else:
            temp[k+keyword] = round(np_files[k].sum())
            temp[k+keyword2] = round(np_files_vol[k] * temp[k+keyword], 6)

    # Dice Score
    # temp['GT-MS_dice'] = dice(np_files['GT'], np_files['MS']) # Ground Truth - Manual Segmentation
    # temp['GT-SC_dice'] = dice(np_files['GT'], np_files['SC']) # Ground Truth - Smart Click Segmentation
    temp['MS_SC_dice'] = dice(np_files['MS'], np_files['SC']) # Manual Segmentation - Smart Click Segmentation


    df = df.append(pd.Series(temp, name=patient_id), ignore_index=True)

DEBUG: list directories - ['WC-IRB1308014251_003', 'WC-IRB1308014251_008', 'WC-IRB1308014251_011', 'WC-IRB1308014251_020', 'WC-IRB1308014251_024_Visit2', 'WC-IRB1308014251_025', 'WC-IRB1308014251_030', 'WC-IRB1308014251_032', 'WC-IRB1308014251_034', 'WC-IRB1308014251_039_Visit1']
WC-IRB1308014251_003 --- {'vol': 0.67734375, 'SC': 0.67734375, 'MS': 0.67734375}
WC-IRB1308014251_008 --- {'vol': 0.9375, 'SC': 0.9375, 'MS': 0.9375}
WC-IRB1308014251_011 --- {'vol': 1.8566349029541016, 'SC': 1.8566349029541016, 'MS': 1.8566349029541016}
WC-IRB1308014251_020 --- {'vol': 1.134375, 'SC': 1.134375, 'MS': 1.134375}
WC-IRB1308014251_024_Visit2 --- {'vol': 0.67734375, 'SC': 0.67734375, 'MS': 0.67734375}
Error:  C:\Users\colli\Desktop\lesionInterfaces\ISMRM\WC-IRB1308014251_025\25.nii.gz
WC-IRB1308014251_025 --- {'vol': 1.0986714363098145, 'SC': 1.0986714363098145, 'MS': 1.0986714363098145}
Error:  C:\Users\colli\Desktop\lesionInterfaces\ISMRM\WC-IRB1308014251_030\DR_liver_lesion_hang_click_vol.nii.g

In [32]:
df

,patient_id,vol_voxCnt,vol_volume(in cm^3),SC_voxCnt,SC_volume(in cm^3),MS_voxCnt,MS_volume(in cm^3),MS_SC_dice
0,WC-IRB1308014251_003,6323200.0,4.282980e+06,4719.0,3196.385156,5862.0,3970.589062,0.888763
1,WC-IRB1308014251_008,5529600.0,5.184000e+06,49033.0,45968.437500,52839.0,49536.562500,0.918820
2,WC-IRB1308014251_011,4456448.0,8.273997e+06,71072.0,131954.755823,74369.0,138076.081098,0.976465
3,WC-IRB1308014251_020,5529600.0,6.272640e+06,9929.0,11263.209375,10851.0,12309.103125,0.955438
4,WC-IRB1308014251_024_Visit2,5824000.0,3.944850e+06,12919.0,8750.603906,19067.0,12914.913281,0.605140
5,WC-IRB1308014251_025,2031616.0,2.232078e+06,11649.0,12798.423562,13167.0,14466.206802,0.937379
6,WC-IRB1308014251_030,5529600.0,4.678560e+06,5158.0,4364.151562,7565.0,6400.699219,0.808772
7,WC-IRB1308014251_032,26214400.0,6.479551e+06,32568.0,8050.003233,41489.0,10255.053554,0.742293
8,WC-IRB1308014251_034,6451200.0,5.457659e+06,9986.0,8448.068423,10738.0,9084.253828,0.685099
9,WC-IRB1308014251_039_Visit1,3538944.0,6.912124e+06,1220.0,2382.855316,2228.0,4351.640692,0.538283


In [33]:
df.to_csv('dice_score.csv')